In [13]:
import pandas as pd
import os.path
import pickle as pk
from math import isnan
from fbri.private.sql.query import execute

def getFilename(domain):
    return (domain+"_domain_stats.tsv")


def getStats(same_month=True):
    database = "fbri_prod_private"
    urls_table = "erc_condor_url_attributes_dp_final_v3"
    breakdown_table = "erc_condor_url_breakdowns_dp_clean_partitioned_v2"
    
    month_clause = ''
    if same_month:
        month_clause = "AND b.year_month = date_format(from_unixtime(u.first_post_time_unix),'%Y-%m')"
    
    sql = f"""
    SELECT u.*, COUNT(b.views) as count,
    SUM(b.views) as sum_views, SUM(b.clicks) as sum_clicks, 
    SUM(b.shares) as sum_shares, SUM(b.likes) as sum_likes, 
    SUM(b.loves) as sum_loves, SUM(b.hahas) as sum_hahas, 
    SUM(b.wows) as sum_wows, SUM(b.sorrys) as sum_sorrys, 
    SUM(b.angers) as sum_angers, SUM(b.comments) as sum_comments, 
    SUM(b.share_without_clicks) as sum_share_without_clicks
    FROM {database}.{breakdown_table} b
    JOIN {database}.{urls_table} u
    ON (
     u.url_rid = b.url_rid
     AND b.c ='GB'
     AND u.public_shares_top_country = 'GB'
     {month_clause}
    ) GROUP BY u.url_rid,u.clean_url,u.full_domain,
    u.parent_domain,u.first_post_time,u.first_post_time_unix,
    u.share_title,u.share_main_blurb,u.tpfc_rating,
    u.tpfc_first_fact_check,u.tpfc_first_fact_check_unix,
    u.spam_usr_feedback,u.false_news_usr_feedback,
    u.hate_speech_usr_feedback,u.public_shares_top_country, u.year_month
    """    
    df = execute(sql)
    
    return (df)

In [14]:
df  = getStats()

Query execution started. Query ID: cd04faf9-440c-4a71-872b-a4b13d09776c
running for 1.5 sec, status: RUNNING
running for 3.75 sec, status: RUNNING
running for 7.12 sec, status: RUNNING
running for 12.19 sec, status: RUNNING
running for 19.78 sec, status: RUNNING
running for 48.26 sec, status: RUNNING
running for 73.89 sec, status: RUNNING
running for 170.0 sec, status: RUNNING
running for 256.49 sec, status: RUNNING


In [17]:
df = df.drop(columns = ['share_title','share_main_blurb'])

In [18]:
df.to_parquet('./data/all_urls_no_title_or_blurb.parquet')